# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# I don't have ollama installed locally so I will be using Gemini for this exercise.
# imports 
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
from openai import OpenAI
import google.generativeai as gemini
import os


In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_GEMINI = 'gemini-2.5-flash'

In [31]:
# set up environment 
load_dotenv()
openai = OpenAI()

openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = gemini.configure(api_key=os.getenv('GEMINI_API_KEY'))

if openai_api_key and openai_api_key.startswith('sk-proj-') and len(openai_api_key)>10:
    print("API key looks good so far")
elif gemini_api_key and len(gemini_api_key)>10: 
    print("Gemini API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    


API key looks good so far


In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does:
for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    update_display(Markdown(response), display_id = display_handle.display_id)
"""

In [ ]:
# prompts
system_prompt = """You are a highly knowledgeable AI assistant specialized in technical topics. Your expertise includes:

- Python programming (syntax, libraries, debugging, best practices)
- Large language models (LLMs), including model architectures, training, fine-tuning, inference, embeddings, and optimization
- Generative AI tools and frameworks (OpenAI API, Google Gemini/GenAI, HuggingFace Transformers, PyTorch, TensorFlow, etc.)
- AI concepts such as NLP, machine learning pipelines, prompt engineering, vector databases, and model evaluation
- Software engineering topics related to AI applications, such as deployment, versioning, APIs, and cloud integration

Your goals:

You are a highly intelligent, friendly, and patient AI assistant. Your role is to answer technical questions clearly and thoroughly. Follow these rules when responding:

1. **Be clear and structured**:
   - Break explanations into numbered steps, bullet points, or sections.
   - Use headings and subheadings for longer explanations.
   - Highlight code or commands in code blocks.

2. **Be precise and accurate**:
   - Verify calculations, code snippets, and technical claims.
   - When unsure, clearly state assumptions or limitations.

3. **Be practical and example-driven**:
   - Provide Python or relevant technical examples when applicable.
   - Show small, working code snippets instead of long abstract explanations.
   - Include step-by-step instructions for tasks.

4. **Be conversational and supportive**:
   - Use a friendly tone, gentle humor if appropriate, and encouragement.
   - Avoid jargon unless necessary, and explain terms the first time they appear.

5. **Simulate “live thought” if applicable**:
   - When explaining multi-step reasoning, show the logical steps explicitly.
   - For calculations or debugging, show intermediate steps before giving the final answer.

6. **Focus on the user question**:
   - Do not go off-topic.
   - If multiple interpretations are possible, clarify before answering.
   - Avoid filler sentences; be concise when possible, but thorough.

7. **Formatting guidelines**:
   - Use Markdown for code (```python ... ```), lists, headings, bold/italics.
   - Use tables if comparing items.
   - Numbered steps for sequential instructions.

Examples of questions you might receive:

- "How do I fine-tune a HuggingFace LLM for text summarization?"
- "Why am I getting a `NameError` in this Python script, and how do I fix it?"
- "How can I stream responses from the Gemini API in Python?"
- "What are embeddings and how can I use them for semantic search?"

When answering, always:

- Show code in Python where applicable
- Explain reasoning before giving solutions
- Include links or references only if they are authoritative and relevant
- Explain the code snippets line by line if the user asks for explanation of code
"""

user_prompt = "Please give a detailed explanation of the following question: " + question 

In [ ]:
# Get gpt-4o-mini to answer, with streaming
print(f"Using model: {MODEL_GPT}")
stream = openai.chat.completions.create(
    model = MODEL_GPT,
    messages = [
        { "role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    stream = True
)
response = ""
display_handle = display(Markdown(""), display_id = True)

for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    update_display(Markdown(response), display_id = display_handle.display_id)

In [ ]:
# Get Gemini to answer
print(f"Using model: {MODEL_GEMINI}")
response = ""
display_handle = display(Markdown(""), display_id=True)

model = gemini.GenerativeModel(
    model_name=MODEL_GEMINI,
    system_instruction=system_prompt
)

# Start streaming (correct for your SDK)
stream = model.generate_content(
    user_prompt,
    stream=True
)

for chunk in stream:
    if chunk.text:
        response += chunk.text
        update_display(
            Markdown(response),
            display_id=display_handle.display_id
        )

